In [1]:
from math import exp
import pyomo.environ as pyo
import pyomo.gdp as gdp
from pyomo.core.expr.logical_expr import atleast
from pyomo.core import TransformationFactory
from pyomo.environ import units

In [2]:
PEAK_CONSUMPTION = 10
"""
https://www.constellation.com/energy-101/energy-education/average-home-power-usage.html#how-many-kwh-house-use
"""

'\nhttps://www.constellation.com/energy-101/energy-education/average-home-power-usage.html#how-many-kwh-house-use\n'

In [40]:
model = pyo.ConcreteModel()

model.peak_load = pyo.Param(units=units.kWh, initialize=60)

model.battery_capacity = pyo.Var(
    within=pyo.NonNegativeReals, units=units.kW, bounds=(0, 100)
)
model.solar_capacity = pyo.Var(
    within=pyo.NonNegativeReals, units=units.kWh, bounds=(0, 10)
)


model.battery_capacity_disjunct = gdp.Disjunct()
model.battery_capacity_disjunct.solar_less_than_peak = pyo.Constraint(
    expr=model.solar_capacity >= model.peak_load
)
model.battery_capacity_disjunct.battery_less_than_excess_solar = pyo.Constraint(
    expr=model.battery_capacity <= model.solar_capacity - model.peak_load
)

model.battery_capacity_Zero_disjunct = gdp.Disjunct()
model.battery_capacity_Zero_disjunct.battery_Zero = pyo.Constraint(
    expr=model.battery_capacity == 0
)

model.either_or_disjunction = gdp.Disjunction(
    expr=[model.battery_capacity_disjunct, model.battery_capacity_Zero_disjunct]
)
"""

model.battery_charging_constraint = pyo.Constraint(
    expr=model.battery_capacity <= model.solar_capacity - model.peak_load
)
"""


TransformationFactory("gdp.bigm").apply_to(model)
optimizer = pyo.SolverFactory("gurobi", logfile="log.log")
optimizer.solve(
    model,  # tee=True
)

print(model.display())

Model unknown

  Variables:
    battery_capacity : Size=1, Index=None, Units=kW
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :   100 : False : False : NonNegativeReals
    solar_capacity : Size=1, Index=None, Units=kWh
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  10.0 :    10 : False : False : NonNegativeReals

  Objectives:
    None

  Constraints:
    None

  Blocks:
    Block _pyomo_gdp_bigm_reformulation
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        either_or_disjunction_xor : Size=1
            Key  : Lower : Body : Upper
            None :   1.0 :  1.0 :   1.0
    
      Blocks:
        Block _pyomo_gdp_bigm_reformulation.relaxedDisjuncts[0]
        
          Variables:
            None
        
          Objectives:
            None
        
          Constraints:
            transformedConstraints : Size=2
                Key          